In [1]:
import sys
import os
if 'jbook' in os.getcwd():
    os.chdir(os.path.abspath(os.path.join("../../..")))
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from scipy import stats
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from myst_nb import glue
import seaborn.objects as so
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from bcd.explore.meta.case import CaseExplorer
from bcd.explore.meta.multivariate.pipeline import PipelineBuilder
from bcd.explore.meta.multivariate.selection import ModelSelector

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
sns.set_style('whitegrid')
sns.set_palette("Blues_r")

In [3]:
CASE_FP = "data/meta/3_cooked/cases.csv"

In [4]:
cases = CaseExplorer(filepath=CASE_FP)
calc = CaseExplorer(df=cases.get_calc_data())
mass = CaseExplorer(df=cases.get_mass_data())

In [5]:
summary = cases.summary()
glue("eda1_summary", summary)

,Summary
Patients,1566
Cases,3566
Calcification Cases,1872
Calcification Cases - Benign,1199
Calcification Cases - Malignant,673
Mass Cases,1694
Mass Cases - Benign,910
Mass Cases - Malignant,784


In [6]:
st = summary.T
pct_calc = round(st['Calcification Cases'] / st['Cases'] * 100,2).values[0]
pct_mass = round(st['Mass Cases'] / st['Cases'] * 100,2).values[0]
pct_calc_mal = round(st['Calcification Cases - Malignant'] / st['Calcification Cases'] * 100,2).values[0]
pct_calc_bn = round(st['Calcification Cases - Benign'] / st['Calcification Cases'] * 100,2).values[0]
pct_mass_mal = round(st['Mass Cases - Malignant'] / st['Mass Cases'] * 100,2).values[0]
pct_mass_bn = round(st['Mass Cases - Benign'] / st['Mass Cases'] * 100,2).values[0]
cases_per_patient = round(st['Cases'] / st['Patients'],2).values[0]

glue("pct_calc", pct_calc)
glue("pct_mass", pct_mass)
glue("pct_calc_mal", pct_calc_mal)
glue("pct_calc_bn", pct_calc_bn)
glue("pct_mass_mal", pct_mass_mal)
glue("pct_mass_bn", pct_mass_bn)
glue("cases_per_patient", cases_per_patient)

52.5

47.5

35.95

64.05

46.28

53.72

2.28

In [7]:
cases.get_calc_data().sample(5)
cases.get_mass_data().sample(5)

,case_id,patient_id,breast_density,left_or_right_breast,image_view,abnormality_id,calc_type,calc_distribution,assessment,pathology,subtlety,fileset,cancer
48,Calcification-Train_P_00031_LEFT_CC_1,P_00031,4,LEFT,CC,1,PLEOMORPHIC,CLUSTERED,4,MALIGNANT,1,train,True
1553,Calcification-Test_P_00041_LEFT_MLO_2,P_00041,1,LEFT,MLO,2,LUCENT_CENTERED,SEGMENTAL,2,BENIGN_WITHOUT_CALLBACK,5,test,False
46,Calcification-Train_P_00030_RIGHT_CC_1,P_00030,2,RIGHT,CC,1,VASCULAR,SEGMENTAL,2,BENIGN_WITHOUT_CALLBACK,4,train,False
1804,Calcification-Test_P_01635_RIGHT_MLO_2,P_01635,2,RIGHT,MLO,2,AMORPHOUS-PLEOMORPHIC,CLUSTERED,4,MALIGNANT,3,test,True
318,Calcification-Train_P_00455_LEFT_CC_1,P_00455,1,LEFT,CC,1,LUCENT_CENTERED,SEGMENTAL,2,BENIGN_WITHOUT_CALLBACK,3,train,False


,case_id,patient_id,breast_density,left_or_right_breast,image_view,abnormality_id,mass_shape,mass_margins,assessment,pathology,subtlety,fileset,cancer
2060,Mass-Train_P_00279_LEFT_CC_1,P_00279,3,LEFT,CC,1,ROUND,OBSCURED,0,BENIGN,4,train,False
1952,Mass-Train_P_00106_LEFT_CC_3,P_00106,1,LEFT,CC,3,LYMPH_NODE,CIRCUMSCRIBED,2,BENIGN_WITHOUT_CALLBACK,4,train,False
2347,Mass-Train_P_00695_RIGHT_CC_1,P_00695,2,RIGHT,CC,1,ASYMMETRIC_BREAST_TISSUE,ILL_DEFINED,5,MALIGNANT,5,train,True
2650,Mass-Train_P_01134_RIGHT_CC_1,P_01134,2,RIGHT,CC,1,IRREGULAR,SPICULATED,5,MALIGNANT,5,train,True
2758,Mass-Train_P_01268_RIGHT_MLO_1,P_01268,3,RIGHT,MLO,1,LOBULATED,OBSCURED,0,BENIGN,5,train,False
